In [63]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.colors as mcolors
import healpy as hp
import math
from shapely.geometry import Point, Polygon
import plotly.graph_objects as go

In [197]:
def main(order, i, num, step):
    num_points = num
    # Create points for sphere
    theta = np.linspace(0, 2 * np.pi, 100)
    phi = np.linspace(0, np.pi, 50)
    theta, phi = np.meshgrid(theta, phi)
    r = 1
    # Convert to Cartesian coordinates
    x = r * np.sin(phi) * np.cos(theta)
    y = r * np.sin(phi) * np.sin(theta)
    z = r * np.cos(phi)
    
    sphere = go.Surface(x=x,y=y,z=z,colorscale='Gray',opacity=.2,showscale=False)
    
    
    points = getBoundaries(order, i, step)
    
    x = points[0] 
    y = points[1]
    z = points[2]
    
   # print(str(x[1]) + " " + str(y[1]) + " " + str(z[1]) )
    # Create scatter3d plot
    scatter = go.Scatter3d(x=x,y=y,z=z,mode='markers',marker=dict(size=3,color="red",colorscale='Viridis',opacity=0.8), name="Borders")
    
    pointForm  =np.transpose(points)
    
    arc12 = pointForm[0: step +1]
    #print(np.shape(arc12))
    arc23 = pointForm[step: 2*step +1]
    arc34 = pointForm[2*step: 3*step +1]
    arc41 = pointForm[3*step:]
    # print(arc41)
    # print(pointForm[0])
    arc41 = np.append(arc41, [pointForm[0]], axis =0 )
    # print(np.shape(arc12))
    # print(np.shape(arc23))
    # print(np.shape(arc34))
    # print(np.shape(arc41))
    arcs = np.concatenate((arc12, arc23, arc34, arc41), axis = 0)
    # print(arcs)
    #p1 = go.Scatter3d(x=x[3*step:], y=y[3*step:],z= z[step*3:],mode='markers', marker=dict(size = 10, color='green',opacity=.8))
    
    
    # arc12, arc23, arc34, arc41 = getArcs(points, num_points)
    # print(arc12[-1, :])
    # arcs = np.concatenate((arc12, arc23, arc34, arc41), axis = 0)
    scatters = go.Scatter3d(x=arcs[:,0],y=arcs[:,1],z=arcs[:,2],mode='markers',marker=dict(size=2,color="blue",colorscale='Viridis',opacity=0.8), name="Borders")
    
    new_points, arc_lengths= getPoints(arc12, arc23, arc34, arc41, step, num_points)
    print(np.shape(new_points))
    x1 = new_points[:,0]
    y1 = new_points[:,1]
    z1 = new_points[:,2]
    # Create scatter3d plot
    scatter1 = go.Scatter3d(x=x1,y=y1,z=z1,mode='markers',marker=dict(size=2,color="yellow",colorscale='Viridis',opacity=0.8), name="Points")
    

    # test1 = go.Scatter3d(x=new_points[150:153, 0], y=new_points[150:153,1],z= new_points[150:153, 2],mode='markers', marker=dict(size = 2, color='green',opacity=.8))
  

    # for j in range(10):
    #     print(new_points[j+150,0], new_points[j+150, 1],new_points[j+150, 2] )
        
        
        
    # p1 = go.Scatter3d(x=new_points[51:52, 0], y=new_points[51:52,1],z= new_points[51:52,2],mode='markers', marker=dict(size = 2, color='green',opacity=.8))
    # p2 = go.Scatter3d(x=new_points[(2598):(2599), 0], y=new_points[(2598):(2599),1],z= new_points[(2598):(2599), 2],mode='markers', marker=dict(size = 2, color='green',opacity=.8))    
    # # test = go.Scatter3d(x=new_points[0:50, 0], y=new_points[0:50,1],z= new_points[0:50, 2],mode='markers', marker=dict(size = 5, color='green',opacity=.8))
    # small_arcs = []
    # for i in range(num_points*num_points):
    #     p1 = new_points[i]
    #     p2 = new_points[(num_points*num_points-1 + num_points) - i//num_points + num_points*(i%num_points)]
    #     small_arcs.append(angle_finder(p1,p2))    
        
    fig = go.Figure(data=[sphere, scatter, scatter1])
    fig.update_layout(
        title=f'HEALPix Points on Sphere (order={order})',
        scene=dict(
            xaxis=dict(showbackground=False, visible=False),
            yaxis=dict(showbackground=False, visible=False),
            zaxis=dict(showbackground=False, visible=False),
            aspectmode='data'
        ),
        width=800,
        height=800
    )
    # Display the plot
    fig.show()
    # return arc_lengths, small_arcs

# t,s  = main(0,0)

In [198]:
main(0,0,50, 49)

(5000, 3)


In [114]:
def getBoundaries(order, i, step):
    x_coor = []
    y_coor = []
    z_coor = []
    # for i in range(hp.order2npix(order)):
    test = hp.boundaries(hp.order2nside(order), i, step=step)
    x_coor.extend(test[0])
    y_coor.extend(test[1])
    z_coor.extend(test[2])

    return np.array((x_coor, y_coor, z_coor))

In [115]:
def getArcs(points, num_points):
    
    p1, p2, p3, p4= np.transpose(points)
    arc12 = great_circle_arc(p1, p2, num_points)
    arc23 = great_circle_arc(p2, p3, num_points)
    arc34 = great_circle_arc(p3, p4, num_points)
    arc41 = great_circle_arc(p4, p1, num_points)
    
    return arc12, arc23, arc34, arc41
    

In [183]:
def getPoints(arc12, arc23, arc34, arc41,step, num_points = 50):
    new_points = []
    all_arc_lengths = []
    
    #This function returns arcs both from top to bottom as well as side to side
    for i in range(step+1):
        arc = great_circle_arc(arc12[i], arc34[step  - i], num_points)
        new_points.extend(arc)
        arc_lengths=[]
        for j in range(num_points-1):
            arc_lengths.append(angle_finder(arc[j], arc[j+1]))
        all_arc_lengths.append(arc_lengths)
    for i in range(step+1):
        arc = great_circle_arc(arc23[i], arc41[step - i], num_points)
        new_points.extend(arc)
        arc_lengths=[]
        for j in range(num_points-1):
            arc_lengths.append(angle_finder(arc[j], arc[j+1]))
        all_arc_lengths.append(arc_lengths)


    

                
                    
    return np.array(new_points), np.array(all_arc_lengths)

In [58]:
def great_circle_arc(p1, p2, num_points=50):
    """Generates points along a great circle arc between two points."""
    p1_norm = p1 / np.linalg.norm(p1)
    p2_norm = p2 / np.linalg.norm(p2)
    points = []
    for i in range(num_points):
        t = i / (num_points - 1)
        # Linear interpolation and normalization
        pt = (1 - t) * p1_norm + t * p2_norm
        pt_norm = pt / np.linalg.norm(pt)
        points.append(pt_norm)
    return np.array(points)

In [59]:
def diff_in_angle(p1, p2):
   new_p1 = spherical(p1)
   new_p2 = spherical(p2)
   theta= abs(math.degrees(new_p1[0]) - math.degrees(new_p2[0]))
   phi = abs(math.degrees(new_p1[1]) - math.degrees(new_p2[1]))
   return theta, phi

In [60]:
def spherical(point):
    return np.arctan2(point[1], point[0]), np.arccos(point[2])

In [61]:
import math

def spherical_angle_diff(point1, point2):
    """
    Calculates the difference in theta and phi angles between two points on a sphere given their Cartesian coordinates.

    Args:
        point1: A tuple of length 3 representing the Cartesian coordinates of the first point.
        point2: A tuple of length 3 representing the Cartesian coordinates of the second point.

    Returns:
        A tuple containing the difference in theta and phi angles.
    """
    
    def to_spherical(point):
        x, y, z = point
        r = math.sqrt(x**2 + y**2 + z**2)
        theta = math.atan2(y, x)
        phi = math.acos(z / r)
        return theta, phi

    theta1, phi1 = to_spherical(point1)
    theta2, phi2 = to_spherical(point2)

    return (theta2 - theta1) *180/np.pi, (phi2 - phi1)*180/np.pi

In [62]:
def angle_finder(point1, point2):

    point1 = np.array(point1)
    point2 = np.array(point2)

    # Calculate the dot product of the two vectors
    dot_product = np.dot(point1, point2)

     # Ensure the value is within the valid range for arccos (-1 to 1)
    cos_theta = np.clip(dot_product, -1, 1)
    
    # Calculate the angle in radians using arccos
    angle = math.degrees(math.acos(cos_theta))

    return angle